## Configurando modelo

In [1]:
! pip install openai


[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


### Testing prompt

In [3]:
questão = """QUESTÃO 126  
Um garoto comprou vários abacates na feira, mas 
descobriu que eles não estavam maduros o suficiente 
para serem consumidos. Sua mãe recomendou que ele 
colocasse os abacates em um recipiente fechado, pois 
isso aceleraria seu amadurecimento. Com certa dúvida, o 
garoto realizou esta experiência: colocou alguns abacates 
no recipiente e deixou os demais em uma fruteira aberta. 
Surpreendendo-se, ele percebeu que os frutos que estavam 
no recipiente fechado amadureceram mais rapidamente.
 A aceleração desse processo é causada por"""
alternativas = """
A acúmulo de gás etileno.
B redução da umidade do ar.
C aumento da concentração de CO2.
D diminuição da intensidade luminosa.
E isolamento do contato com O2 atmosférico."""

In [4]:
# This example is the new way to use the OpenAI lib for python
from openai import OpenAI

client = OpenAI(
api_key = "LL-8TEmrzbYgkmK420Q2gMNmvzoNL2JhY7rNyp62M0qzqyuUvwfvMTIHVQxJI6TeVn9",
base_url = "https://api.llama-api.com"
)

conteudo = "Você deve responder perguntas em português sobre diversos temas. Suas respostas devem sem compostas com APENAS a letra da alternativa correta, não retorne mais nada para o user. Answer only with the character of the correct alternative. A,B,C,D or E."

response = client.chat.completions.create(
    model="llama-70b-chat",
    max_tokens= 30,
    messages=[
        {"role": "system", "content": "You are a helpful assistant that should answer questions in Portuguese about various topics. Completing the phrase: A resposta correta é:. You must not answer more than the letter of the alternative"},
        {"role": "user", "content": f"com base nesse enunciado: {questão} \n responda qual a letra da alternativa correta, entre as opções a seguir: \n {alternativas} \n a alternativa correta é:"}
    ],
    
)

#print(response)
print(response.model_dump_json(indent=2))
print(response.choices[0].message.content)

{
  "id": null,
  "choices": [
    {
      "finish_reason": "max_token",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "A resposta correta é: A acúmulo de gás etileno. \n\nExplanation: \nO",
        "role": "assistant",
        "function_call": null,
        "tool_calls": null
      }
    }
  ],
  "created": 1709908271,
  "model": "llama-70b-chat",
  "object": null,
  "system_fingerprint": null,
  "usage": {
    "completion_tokens": 24,
    "prompt_tokens": 294,
    "total_tokens": 318
  }
}
A resposta correta é: A acúmulo de gás etileno. 

Explanation: 
O


### Experiment

In [ ]:
from openai import OpenAI

context = "You are going to answer questions in Portuguese about various topics. Your answers must be composed with ONLY the letter of the correct alternative, do not return anything else to the user, such as explanation and text, only the letter. Completing the phrase:  A resposta correta é: "

def submit_questions(prompt):    
    
    client = OpenAI(
        api_key = "LL-8TEmrzbYgkmK420Q2gMNmvzoNL2JhY7rNyp62M0qzqyuUvwfvMTIHVQxJI6TeVn9",
        base_url = "https://api.llama-api.com"
    )
    
    response = client.chat.completions.create(
        model="llama-70b-chat",
        max_tokens= 30,
        messages=[
            {"role": "system", "content": f'{context}'},
            {"role": "user", "content": f'{prompt}'}
        ],
        
    )
    # print(response.choices[0].message.content.replace('\n', ''))

    if response.choices[0].message.content == "" or response.choices[0].message.content == None or len(response.choices[0].message.content) < 3:
        raise Exception("A resposta está vazia")
    
    if ( not " A" in response.choices[0].message.content) and ( not "B" in response.choices[0].message.content) and ( not "C" in response.choices[0].message.content) and ( not "D" in response.choices[0].message.content) and ( not "E" in response.choices[0].message.content):
        raise Exception("A resposta não contém a letra da alternativa correta")
   
    return response.choices[0].message.content.replace('\n', '')

In [ ]:
CSVS_PATH = '../TCC/csvs'
EXPERIMENTS_PATH = '../TCC/experiments'
EXPERIMENT_NUMBER = 1

import pandas as pd
import os

years = [2023]

for year in years:
    
    enem_csv = pd.read_csv(f'{CSVS_PATH}/enem_{year}_questions.csv').copy()
    model_answers = ''    
    problematic_questions = []
    problematic_outputs = []
    
    for index, row in enem_csv.iterrows():
        
        question = row['body']
        alternatives = row['alternatives']
        prompt = f'usando como base esse enunciado: {question} \n responda qual a alternativa correta, entre as opções a seguir: \n {alternatives} \n a alternativa correta é:'
        teste = ''
        
        try:
            
            teste = submit_questions(prompt)
            print(teste)
            
        except:
            problematic_questions.append(row['id'])
            
            with open(f'{EXPERIMENTS_PATH}/{EXPERIMENT_NUMBER}/{year}/Llama2_{year}_answers.txt', 'w+',encoding='utf-8') as f:
                f.write(model_answers)
                
        model_answers += f'{row["id"]}\n{teste}\n'
        
    
    model_answers += f'\nQuestões problemáticas: {problematic_questions}'
    model_answers += f'\nSaídas problemáticas: {problematic_outputs}'
    
    with open(f'{EXPERIMENTS_PATH}/{EXPERIMENT_NUMBER}/{year}/Llama2_{year}_answers.txt', 'w+',encoding='utf-8') as f:
        f.write(model_answers)